# Capstone Project

## Introduction


### Description
Machine Learning is now being used in many fields for solving several problems and have benefited the data scientists in several fields in terms of accuracy, cost, and time. In this aspect, we are using machine learning to predict the severity of car accidents on the basis of the type of accident.

### Data 

The data we have on hand is quite extensive, providing us with relevant attributes such as the location and time of the accident, as well as the consequent weather and lighting conditions when the event occurred. There are also details indicating how many pedestrians were involved, or if one or more cars that were affected were parked or not.

### Data benefits:
By looking at variables such as :

<li> road condition </li>
<li>light condition</li>
<li>weather</li>
<li>speed</li>
<li>lane</li>
we can build a model to see if the variables above mentioned will affect severity of traffic accident.

Subsequently, we will do data cleaning, feature selection and data normalization before we actually apply machine learning algorithms.

For Data cleaning:


<li>INCKEY, INTKEY, and COLDETKEY- It is unnecessary information</li>
<li>REPORTNO, EXCEPTTRSNCODE, EXCEPTRSNDESC- Without any description, the data is useless</li>
<li>Have to remove the nan or null values</li>
<li>Have to remove the blank or 0's variables</li>

This model can classify the severity of the accident which would provide the driver with the 'worst-case scenario', rather than a probabilistic estimate of an accident occuring. This will help in inducing an appropriate level of cautiousness in the driver!

Features used are speeding and road condition for this model.

## Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression

print("Done")

Done


In [2]:
!wget -O Data-Collisions.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv

--2020-09-07 14:29:40--  https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
Resolving s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73917638 (70M) [text/csv]
Saving to: ‘Data-Collisions.csv’

100%[======================================>] 73,917,638  35.0MB/s   in 2.0s   

2020-09-07 14:29:43 (35.0 MB/s) - ‘Data-Collisions.csv’ saved [73917638/73917638]



In [4]:
df = pd.read_csv("Data-Collisions.csv")

select = ['SEVERITYCODE', 'SPEEDING','ROADCOND']
df = df[select]

for feature in ["SPEEDING", "ROADCOND"]:
    print(df[feature].unique())
df.head()

[nan 'Y']
['Wet' 'Dry' nan 'Unknown' 'Snow/Slush' 'Ice' 'Other' 'Sand/Mud/Dirt'
 'Standing Water' 'Oil']


,SEVERITYCODE,SPEEDING,ROADCOND
0,2,NaN,Wet
1,1,NaN,Wet
2,1,NaN,Dry
3,1,NaN,Dry
4,2,NaN,Wet


### Assumptions for missing data

<li>Speeding is unknown</li>
<li>The road condition is unknown</li>

In [5]:
df['SPEEDING'] = df['SPEEDING'].fillna('N')

df['ROADCOND'] = df['ROADCOND'].fillna('Unknown')


for feature in ["SPEEDING", "ROADCOND"]:
    print(df[feature].unique())

['N' 'Y']
['Wet' 'Dry' 'Unknown' 'Snow/Slush' 'Ice' 'Other' 'Sand/Mud/Dirt'
 'Standing Water' 'Oil']


In [6]:
df.shape

(194673, 3)

### Assumptions for data analysis

We assume that the road conditions have the only options of being dry, wet and other.

In [7]:
df['ROADCOND'].replace(to_replace=['Wet','Dry','Unknown','Snow/Slush','Ice','Other','Sand/Mud/Dirt','Standing Water','Oil'], value = ['Bad','Good','Good','Bad','Bad','Good','Bad','Bad','Bad'], inplace=True)

In [8]:
df["SPEEDING"].replace(to_replace=['N', 'Y'], value=[0,1], inplace=True)
df['ROADCOND'].replace(to_replace=['Good','Bad'],value=[0,1],inplace=True)
condition = df[['SPEEDING','ROADCOND']]
condition.head()

,SPEEDING,ROADCOND
0,0,1
1,0,1
2,0,0
3,0,0
4,0,1


## Data Anaysis

To check whether speed and road conditions are bad

For seeing the effect of speeding on accidents:

In [9]:
speed_observation = df.groupby(['SPEEDING'])['SEVERITYCODE'].value_counts(normalize=True)
speed_observation

SPEEDING  SEVERITYCODE
0         1               0.705099
          2               0.294901
1         1               0.621665
          2               0.378335
Name: SEVERITYCODE, dtype: float64

In [10]:
road_observation = df.groupby(['ROADCOND'])['SEVERITYCODE'].value_counts(normalize=True)
road_observation

ROADCOND  SEVERITYCODE
0         1               0.710389
          2               0.289611
1         1               0.674176
          2               0.325824
Name: SEVERITYCODE, dtype: float64

This shows that these factors do contribute in the severity of accidents

## Methodology

First we train the model:

In [11]:
x = condition
y = df['SEVERITYCODE'].values.astype(str)
x = preprocessing.StandardScaler().fit(x).transform(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)


print("Training set: ", x_train.shape, y_train.shape)
print("Testing set: ", x_test.shape, y_test.shape)

Training set:  (155738, 2) (155738,)
Testing set:  (38935, 2) (38935,)


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  app.launch_new_instance()


## Method 1

In [ ]:
from sklearn import metrics
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(x_train,y_train)
    yhat=neigh.predict(x_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

mean_acc

In [12]:
KNN_model = KNeighborsClassifier(n_neighbors = 4).fit(x_train, y_train)
predicted = KNN_model.predict(x_test)
KNN_f1 = f1_score(y_test, predicted, average='weighted')
KNN_acc = accuracy_score(y_test, predicted)

## Method 2


In [14]:
Tree_model = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
Tree_model.fit(x_train, y_train)
predicted = Tree_model.predict(x_test)
Tree_f1 = f1_score(y_test, predicted, average='weighted')
Tree_acc = accuracy_score(y_test, predicted)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Method 3

In [15]:
LR_model = LogisticRegression(C=0.01, solver='liblinear').fit(x_train, y_train)
predicted = LR_model.predict(x_test)

LR_f1 = f1_score(y_test, predicted, average='weighted')
LR_acc = accuracy_score(y_test, predicted)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Results

In [16]:
table = {
    "Algorithm": ["KNN", "Decision Tree", "LogisticRegression"],
    "F1-score": [KNN_f1, Tree_f1, LR_f1],
    "Accuracy": [KNN_acc, Tree_acc, LR_acc]
}

table = pd.DataFrame(table)
table

,Algorithm,F1-score,Accuracy
0,KNN,0.591378,0.696751
1,Decision Tree,0.576051,0.699679
2,LogisticRegression,0.576051,0.699679


In [17]:
table = {
    "Intercept": LR_model.intercept_,
    "Coef:SPEEDING ": LR_model.coef_[:,0],
    "Coef:ROADCOND ": LR_model.coef_[:,1],
}

table = pd.DataFrame(table)
table

,Intercept,Coef:SPEEDING,Coef:ROADCOND
0,-0.853729,0.067702,0.068295


We see that the coeffecients are positive indicating that the two factors (speeding and road conditions) do affect the severity of and contribute in the increased chance of an accident

## Conclusion

This model here provides sufficient evidence against speeding and road conditions